Recreation of the hello world problem found in the resources collection.  
Goal is to find a collection of 2 items out of 3, with minimal added values.  

Values: $ v_1=42, v_2=136, v_3=55, v_4=3 $

Goal function:

$$
\min_{(x_1,\dots x_4)\in{0,1}^4} \sum_{i}v_ix_i
$$

Constraint:

$$
\sum_i x_i = 2
$$
QUBO friendly:

$$ 
(2 - \sum_ix_i)^2
$$

Hamiltonian
$$
\begin{align*}
H   &= A*\sum_{i}v_ix_i + B*(2 - \sum_ix_i)^2\\
    &= A*\sum_{i}v_ix_i - 4B \sum_ix_i + B\sum_ix_i +2B\sum_{i,j;i<j}x_ix_j + 4\\
    &= A\sum_{i}v_ix_i - 3B\sum_ix_i + 2B\sum_{i,j;i<j}x_ix_j + 4B\\
    &= (42A-3B)x_1 + (136A-3B)x_2 + (55A-3B)x_2 + (3A-3B)x_4 + 2B\sum_{i,j;i<j}x_ix_j + 4B 
\end{align*}
$$

Because auf the large difference in values in can occur that ignoring the constraint gives a better overall energy by just choosing the one value very low. $A$ and $B$ have to be chosen in a way that this is compensated.  

The maximal energy penalty from $ H_B $ for braking the constaraint by one qbit is $ \Delta H_B = B $. The minimal energy won by $H_A$ for this is $\Delta H_A=42A$. So to be sure we need

$$ 0<\Delta H_A \leq \Delta H_B $$ 
so in the end

$$ B\geq 42A $$

In [16]:
import dimod
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
from dwave.preprocessing import roof_duality

In [36]:
A=5
B=43*A

linear = {1: 42*A-3*B, 2: 136*A-3*B, 3: 55*A-3*B, 4: 3*A-3*B}

quadratic = {(1,2):2*B, (1,3):2*B, (1,4):2*B, (2,3):2*B, (2,4):2*B, (3,4):2*B}

bqm = dimod.BinaryQuadraticModel(linear, quadratic, offset=4*B, vartype=dimod.BINARY)
roof_duality(bqm)

(142.5, {2: 0})

In [37]:
sampler = EmbeddingComposite(DWaveSampler())
result = sampler.sample(bqm, num_reads = 1000)

print('Results from D-Wave')
print(result)

Results from D-Wave
   1  2  3  4 energy num_oc. chain_.
0  1  0  0  1  225.0     620     0.0
1  0  0  0  1  230.0     339     0.0
2  0  0  1  1  290.0      39     0.0
3  1  0  0  0  425.0       2     0.0
['BINARY', 4 rows, 1000 samples, 4 variables]
